In [2]:
import openai
from dotenv import dotenv_values
config = dotenv_values(".env")
openai.api_key = config["OPENAI_API_KEY"]

In [32]:
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pickle
import tiktoken

## Load The Movie Data

In [4]:
dataset_path = "./movie_plots.csv"
df = pd.read_csv(dataset_path)

In [109]:
# Narrow our data set to 5000 recent American movies (to save money)
movies = df[df["Origin/Ethnicity"] == "American"].sort_values("Release Year", ascending=False).head(5000)

In [40]:
# Extract the movie plots into a list
movie_plots = movies["Plot"].values

## Generating The Embeddings

In [19]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):

    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    return openai.Embedding.create(input=text, model=model)["data"][0]["embedding"]

In [41]:
enc = tiktoken.encoding_for_model("text-embedding-ada-002")

In [45]:
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])

In [53]:
total_tokens
cost = total_tokens * (.0004 / 1000)
print(f"Estimated cost ${cost:.2f}")

Estimated cost $1.45


In [57]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "movie_embeddings_cache2.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string,
    model="text-embedding-ada-002",
    embedding_cache=embedding_cache
):
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPENAI FOR {string[:20]}")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [110]:
# This line actaully generates the embeddings
plot_embeddings = [embedding_from_string(plot, model="text-embedding-ada-002") for plot in movie_plots]

## Plot The Embeddings Using Atlas

In [71]:
data = movies[["Title", "Genre"]].to_dict("records")

In [63]:
from nomic import atlas

In [73]:
project = atlas.map_embeddings(
    embeddings=np.array(plot_embeddings),
    data=data
)

2023-04-13 20:37:15.632 | INFO     | nomic.project:_create_project:965 - Creating project `flippant-loft` in organization `coltsteele1`
2023-04-13 20:37:16.774 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
4it [00:09,  2.34s/it]                       
2023-04-13 20:37:26.243 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-04-13 20:37:26.243 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-04-13 20:37:27.083 | INFO     | nomic.project:create_index:1282 - Created map `flippant-loft` in project `flippant-loft`: https://atlas.nomic.ai/map/5899fca0-27a9-40c7-9a65-0c962306e065/469708b0-74ef-4839-91bc-7186f85f13d7
2023-04-13 20:37:27.083 | INFO     | nomic.atlas:map_embeddings:132 - flippant-loft: https://atlas.nomic.ai/map/5899fca0-27a9-40c7-9a65-0c962306e065/469708b0-74ef-4839-91bc-7186f85f13d7


## Reccommending Movies By Plot

In [83]:
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

In [112]:
def print_recommendations_from_strings(
    strings,
    index_of_source_string,
    k_nearest_neighbors=3,
    model="text-embedding-ada-002"
):
    #Get all of the embeddings
    embeddings = [embedding_from_string(string) for string in strings]
    # get embedding for our specific query string
    query_embedding = embeddings[index_of_source_string]
    # get distances between our embedding and all other embeddings
    distances = distances_from_embeddings(query_embedding, embeddings)
    # get indices of the nearest neighbors
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
    
    query_string = strings[index_of_source_string]
    match_count = 0
    for i in indices_of_nearest_neighbors:
        if query_string == strings[i]:
            continue
        if match_count >= k_nearest_neighbors:
            break
        match_count += 1
        print(f"Found {match_count} closest match: ")
        print(f"Distance of: {distances[i]} ")
        print(strings[i])

In [89]:
print_recommendations_from_strings(movie_plots, 2)

Found 1 closest match: 
Distance of: 0.08445958228190453 
As a spacecraft departs a planet, a humanoid alien drinks an iridescent liquid and then dissolves. The remains of the alien cascade into a waterfall. The alien's DNA strands mix with the water.
In 2089, archaeologists Elizabeth Shaw and Charlie Holloway discover a star map in Scotland that matches others from several unconnected ancient cultures. They interpret this as an invitation from humanity's forerunners, the "Engineers". Peter Weyland, the elderly CEO of Weyland Corporation, funds an expedition, aboard the scientific vessel Prometheus, to follow the map to the distant moon LV-223. The ship's crew travels in stasis while the android David monitors their voyage. Arriving in December 2093, mission-director Meredith Vickers informs them of their mission to find the Engineers and not to make contact without her permission.
The Prometheus lands on the barren, mountainous surface near a large, artificial structure, which a team 